# 1. Parameters

In [1]:
# Defaults
cases_dir = 'cases/unset'
reference_file = 'references/NC_045512.gbk.gz'
input_files_all = 'input/input-files.tsv'
iterations = 3
mincov = 10
ncores = 32
number_samples = 10

In [2]:
# Parameters
cases_dir = "cases/case-100"
iterations = 3
number_samples = 100


In [3]:
from pathlib import Path
from shutil import rmtree
from os import makedirs
import imp
fp, pathname, description = imp.find_module('gdi_benchmark', ['../../lib'])
gdi_benchmark = imp.load_module('gdi_benchmark', fp, pathname, description)

cases_dir_path = Path(cases_dir)

if cases_dir_path.exists():
    rmtree(cases_dir_path)
    
if not cases_dir_path.exists():
    makedirs(cases_dir_path)

input_files_all = Path(input_files_all)
reference_file = Path(reference_file)

case_name = str(cases_dir_path.name)
reference_name = reference_file.name.split('.')[0]

cases_input = cases_dir_path / 'input-files-case.tsv'
index_path = cases_dir_path / 'index'
benchmark_path = cases_dir_path / 'index-info.tsv'
output_tree = cases_dir_path / 'tree.tre'

# 2. Create subset input

In [4]:
import pandas as pd

all_input_df = pd.read_csv(input_files_all, sep='\t')
all_input_total = len(all_input_df)

subset_input_df = all_input_df.head(number_samples)
subset_input_total = len(subset_input_df)

subset_input_df.to_csv(cases_input, sep='\t', index=False)

print(f'Wrote {subset_input_total}/{all_input_total} samples to {cases_input}')

Wrote 100/10000 samples to cases/case-100/input-files-case.tsv


# 2. Index genomes

In [5]:
!gdi --version

gdi, version 0.3.0.dev11


## 2.1. Index reads

In [6]:
results_handler = gdi_benchmark.BenchmarkResultsHandler(name=case_name)
benchmarker = gdi_benchmark.IndexBenchmarker(benchmark_results_handler=results_handler,
                                             index_path=index_path, input_files_file=cases_input,
                                             reference_file=reference_file, mincov=mincov, build_tree=True,
                                             ncores=ncores)

benchmark_df = benchmarker.benchmark(iterations=iterations)


Iteration 1 of index/analysis of 100 samples with 32 cores
Removing any extra snakemake directories: ['snakemake-assemblies.1631070699.3527076']


Creating new index: [gdi init cases/case-100/index]


Creating a new index took 2.76 seconds
Analysis running: [gdi --project-dir cases/case-100/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-100/input-files-case.tsv]


Analysis took 2.68 minutes
Index running: [gdi --project-dir cases/case-100/index --ncores 32 load vcf-kmer --reference-file references/NC_045512.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/sars-cov-2/snakemake-assemblies.1631071175.2822504/gdi-input.fofn]


Indexing took 0.16 minutes
Building tree: [gdi --project-dir cases/case-100/index --ncores 32 rebuild tree --align-type full --extra-params '--fast -m GTR+F+R4' NC_045512]


...

A reminder to myself to look for a Python solution for directory sizes (instead of running `du`)



Iteration 2 of index/analysis of 100 samples with 32 cores
Removing any extra snakemake directories: ['snakemake-assemblies.1631071175.2822504']


Removing any existing indexes cases/case-100/index
Creating new index: [gdi init cases/case-100/index]


Creating a new index took 2.77 seconds
Analysis running: [gdi --project-dir cases/case-100/index --ncores 32 analysis --use-conda --no-load-data --reference-file references/NC_045512.gbk.gz --kmer-size 31 --kmer-size 51 --kmer-size 71 --include-kmer --reads-mincov 10 --input-structured-genomes-file cases/case-100/input-files-case.tsv]


Analysis took 2.83 minutes
Index running: [gdi --project-dir cases/case-100/index --ncores 32 load vcf-kmer --reference-file references/NC_045512.gbk.gz /home/CSCScience.ca/apetkau/workspace/genomics-data-index-evaluation/evaluations/sars-cov-2/snakemake-assemblies.1631072360.1355898/gdi-input.fofn]


In [ ]:
benchmark_df

In [ ]:
benchmark_df.to_csv(benchmark_path, sep='\t', index=False)

# 3. Export trees

In [ ]:
!gdi --project-dir {index_path} export tree {reference_name} > {output_tree}
print(f'Wrote tree to {output_tree}')